# Backtesting using VectorBT

source: [Vectorbt GitHub](https://github.com/polakowo/vectorbt).

Vectorbt is a powerful Python library designed for efficient backtesting and analysis of quantitative trading strategies. It stands out for several reasons:
- **Vectorized Operations:** Leveraging NumPy and Pandas, Vectorbt utilizes vectorized operations for faster and more efficient computations, enhancing the speed of backtesting.
- **Flexible Data Management:** It offers a flexible and user-friendly interface for handling financial time series data, allowing users to easily manipulate and preprocess data for analysis.
- **Modular Components:** Vectorbt is built with a modular structure, enabling users to customize and extend functionalities based on their specific needs. This modularity promotes code reusability and scalability.
- **Performance Metrics:** The library provides a comprehensive set of performance metrics, enabling traders to assess the profitability and risk of their strategies thoroughly.
- **Interactive Visualizations:** With built-in plotting capabilities using Plotly and Matplotlib, Vectorbt allows for interactive and customizable visualizations, aiding in the interpretation of backtesting results.
- **Strategy Composition:** Users can easily compose complex trading strategies by combining various signals, indicators, and rules in a concise and expressive manner.
- **Risk Management:** Vectorbt includes tools for implementing robust risk management strategies, essential for controlling drawdowns and maximizing long-term returns.
- **Event-Driven Backtesting:** It supports event-driven backtesting, allowing users to model and test strategies based on specific market events, providing a more realistic simulation.




### Basic Usage

In [1]:
import vectorbt as vbt 
import datetime 
import numpy as np

# end date 
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=2*365)

# set parameters for RSI
rsi_level_lower = 30
rsi_level_upper = 70


# download data for microsoft
micro_price = vbt.YFData.download('MSFT', start=start_date, end=end_date,missing_index='drop').get('Close')


# create RSI
rsi = vbt.RSI.run(micro_price, 14)

# create entries and exits 
entries  = rsi.rsi_crossed_below(rsi_level_lower)
exits = rsi.rsi_crossed_above(rsi_level_upper)

# create portfolio
pf = vbt.Portfolio.from_signals(micro_price, entries, exits, freq='1D')

# print stats
print(pf.stats())

# show portfolio
pf.plot().show()


Start                         2022-01-11 05:00:00+00:00
End                           2024-01-10 05:00:00+00:00
Period                                502 days 00:00:00
Start Value                                       100.0
End Value                                    142.015377
Total Return [%]                              42.015377
Benchmark Return [%]                          23.748315
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              19.651236
Max Drawdown Duration                 117 days 00:00:00
Total Trades                                          8
Total Closed Trades                                   8
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                       75.0
Best Trade [%]                                12.153316
Worst Trade [%]                               -6

### Custom indicators

In [2]:
# define a custom indicator
def custom_idc(close,rsi_window=14,ma_window=50,rsi_level_upper=70,rsi_level_lower=30):

    # close is the input data in numpy array and has number of columns equal to number of assets

    # rsi calculation
    rsi = vbt.RSI.run(close,window=rsi_window).rsi.to_numpy()

    # moving average
    ma = vbt.MA.run(close,window=ma_window).ma.to_numpy()

    # create signal sell=-1,hold=0, buy=1

    # sell if rsi>rsi_level_upper
    trend = np.where(rsi>rsi_level_upper,-1,0)

    # buy if rsi<rsi_level_lower and close>ma
    trend = np.where((rsi<rsi_level_lower)&(close<ma) ,1,trend)

    return trend

# setup indicator
ind = (vbt.IndicatorFactory(
    class_name='Combination',
    short_name='comb',
    input_names=['close'],
    param_names=['rsi_window','ma_window','rsi_level_upper','rsi_level_lower'],
    output_names=['value'])
    .from_apply_func(custom_idc,rsi_window=14,ma_window=50,rsi_level_upper=70,rsi_level_lower=30))


res = ind.run(micro_price,rsi_window=14,ma_window=60,rsi_level_upper=rsi_level_upper,rsi_level_lower=rsi_level_lower)

entries = res.value == 1
exits = res.value == -1

# create portfolio
pf = vbt.Portfolio.from_signals(micro_price, entries, exits, freq='1D')

# print stats
print(pf.stats())

# show portfolio
pf.plot().show()



Start                         2022-01-11 05:00:00+00:00
End                           2024-01-10 05:00:00+00:00
Period                                502 days 00:00:00
Start Value                                       100.0
End Value                                    128.106358
Total Return [%]                              28.106358
Benchmark Return [%]                          23.748315
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                              19.651236
Max Drawdown Duration                 117 days 00:00:00
Total Trades                                          7
Total Closed Trades                                   7
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  71.428571
Best Trade [%]                                12.153316
Worst Trade [%]                               -6

Of course as is evident this simple strategy is not very succesfull. It has major drawdown periods and less returns.